In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

#read files
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

#combine datasets
train['file']='train'
test['file']='test'
data = pd.concat([train, test],ignore_index=True)
data.head()

/Users/trevor/python-projects/analytics/venv/lib/python2.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type,file
0,Low Fat,FDA15,249.8092,3735.1380,Dairy,0.016047,9.30,1999,OUT049,Tier 1,Medium,Supermarket Type1,train
1,Regular,DRC01,48.2692,443.4228,Soft Drinks,0.019278,5.92,2009,OUT018,Tier 3,Medium,Supermarket Type2,train
2,Low Fat,FDN15,141.6180,2097.2700,Meat,0.016760,17.50,1999,OUT049,Tier 1,Medium,Supermarket Type1,train
3,Regular,FDX07,182.0950,732.3800,Fruits and Vegetables,0.000000,19.20,1998,OUT010,Tier 3,NaN,Grocery Store,train
4,Low Fat,NCD19,53.8614,994.7052,Household,0.000000,8.93,1987,OUT013,Tier 3,High,Supermarket Type1,train


In [3]:
#Item_Weight, Outlet_Size, Item_Outlet_Sales have null values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 13 columns):
Item_Fat_Content             14204 non-null object
Item_Identifier              14204 non-null object
Item_MRP                     14204 non-null float64
Item_Outlet_Sales            8523 non-null float64
Item_Type                    14204 non-null object
Item_Visibility              14204 non-null float64
Item_Weight                  11765 non-null float64
Outlet_Establishment_Year    14204 non-null int64
Outlet_Identifier            14204 non-null object
Outlet_Location_Type         14204 non-null object
Outlet_Size                  10188 non-null object
Outlet_Type                  14204 non-null object
file                         14204 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 1.4+ MB


In [4]:
data.Item_Weight.isnull().sum()

2439

In [5]:
#Item_Weight null values to average weight per item
item_avg_weight = data.pivot_table(values='Item_Weight', index='Item_Identifier')
miss_bool = data['Item_Weight'].isnull()
data.loc[miss_bool,'Item_Weight'] = data.loc[miss_bool,'Item_Identifier'].apply(lambda x: item_avg_weight.loc[x].values[0])

In [6]:
data.Item_Weight.isnull().sum()

0

In [7]:
data['Outlet_Size'].unique()

array(['Medium', nan, 'High', 'Small'], dtype=object)

In [8]:
#number of null values in Outlet_Size
data.Outlet_Size.isnull().sum()

4016

In [9]:
outlet_type_by_size = pd.crosstab(data['Outlet_Size'],data['Outlet_Type'])
outlet_type_by_size

Outlet_Type,Grocery Store,Supermarket Type1,Supermarket Type2,Supermarket Type3
Outlet_Size,,,,
High,0,1553,0,0
Medium,0,1550,1546,1559
Small,880,3100,0,0


In [10]:
mapping = {'Grocery Store':'Small'}
mapping2 = data.Outlet_Type.map(mapping)
data['Outlet_Size'] = data.Outlet_Size.combine_first(mapping2)

In [11]:
data.Outlet_Size.isnull().sum()

3091

In [12]:
outlet_location_by_size = pd.crosstab(data['Outlet_Size'],data['Outlet_Location_Type'])
outlet_location_by_size

Outlet_Location_Type,Tier 1,Tier 2,Tier 3
Outlet_Size,,,
High,0,0,1553
Medium,1550,0,3105
Small,2430,1550,925


In [13]:
mapping3 = {'Tier 2':'Small'}
mapping4 = data.Outlet_Location_Type.map(mapping3)
data['Outlet_Size'] = data.Outlet_Size.combine_first(mapping4)

In [14]:
data.Outlet_Size.isnull().sum()

0

In [15]:
#Item_Visibility has a min of 0
data.describe()

,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Establishment_Year
count,14204.000000,8523.000000,14204.000000,14204.000000,14204.000000
mean,141.004977,2181.288914,0.065953,12.793380,1997.830681
std,62.086938,1706.499616,0.051459,4.651716,8.371664
min,31.290000,33.290000,0.000000,4.555000,1985.000000
25%,94.012000,834.247400,0.027036,8.710000,1987.000000
50%,142.247000,1794.331000,0.054021,12.600000,1999.000000
75%,185.855600,3101.296400,0.094037,16.750000,2004.000000
max,266.888400,13086.964800,0.328391,21.350000,2009.000000


In [16]:
data.Item_Visibility.value_counts()

0.000000    879
0.076856      3
0.076841      3
0.077290      3
0.077169      3
0.076975      3
0.077011      3
0.076792      3
0.076483      3
0.046899      2
0.159844      2
0.135708      2
0.135944      2
0.072298      2
0.136008      2
0.136896      2
0.203401      2
0.081788      2
0.075049      2
0.058369      2
0.104784      2
0.080131      2
0.072411      2
0.024635      2
0.052069      2
0.079968      2
0.179192      2
0.079806      2
0.121767      2
0.080625      2
           ... 
0.013147      1
0.098790      1
0.073397      1
0.137756      1
0.017937      1
0.105126      1
0.044156      1
0.141639      1
0.094411      1
0.074883      1
0.039236      1
0.044497      1
0.051147      1
0.175061      1
0.039034      1
0.077508      1
0.078943      1
0.077170      1
0.042959      1
0.013173      1
0.069939      1
0.011305      1
0.069042      1
0.012216      1
0.064142      1
0.209684      1
0.019592      1
0.013530      1
0.008772      1
0.066817      1
Name: Item_Visibility, L

In [17]:
data['Item_Visibility'].replace(0,np.NaN,inplace=True)
data.groupby('Item_Identifier').transform(lambda x: x.fillna(x.mean()))

,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Establishment_Year
0,249.8092,3735.138000,0.016047,9.300,1999
1,48.2692,443.422800,0.019278,5.920,2009
2,141.6180,2097.270000,0.016760,17.500,1999
3,182.0950,732.380000,0.022930,19.200,1998
4,53.8614,994.705200,0.014670,8.930,1987
5,51.4008,556.608800,0.091294,10.395,2009
6,57.6588,343.552800,0.012741,13.650,1987
7,107.7622,4022.763600,0.127470,19.000,1985
8,96.9726,1076.598600,0.016687,16.200,2002
9,187.8214,4710.535000,0.094450,19.200,2007


In [18]:
data.describe()

,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Establishment_Year
count,14204.000000,8523.000000,13325.000000,14204.000000,14204.000000
mean,141.004977,2181.288914,0.070303,12.793380,1997.830681
std,62.086938,1706.499616,0.050168,4.651716,8.371664
min,31.290000,33.290000,0.003575,4.555000,1985.000000
25%,94.012000,834.247400,0.031396,8.710000,1987.000000
50%,142.247000,1794.331000,0.057851,12.600000,1999.000000
75%,185.855600,3101.296400,0.097618,16.750000,2004.000000
max,266.888400,13086.964800,0.328391,21.350000,2009.000000


In [19]:
#Item_Fat_Content dimension is populating with miscoded values
data['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [20]:
Item_Fat_Content_Counts = pd.DataFrame({'count':data.groupby(['Item_Fat_Content']).size()}).reset_index()
Item_Fat_Content_Counts

,Item_Fat_Content,count
0,LF,522
1,Low Fat,8485
2,Regular,4824
3,low fat,178
4,reg,195


In [21]:
data['Item_Fat_Content'].replace(['low fat','LF','reg'],['Low Fat','Low Fat','Regular'], inplace=True)
Item_Fat_Content_Counts = pd.DataFrame({'count':data.groupby(['Item_Fat_Content']).size()}).reset_index()
Item_Fat_Content_Counts

,Item_Fat_Content,count
0,Low Fat,9185
1,Regular,5019


In [22]:
#New Dimension
data['Years_Open'] = 2013 - data['Outlet_Establishment_Year']
data['Years_Open'].describe()

count    14204.000000
mean        15.169319
std          8.371664
min          4.000000
25%          9.000000
50%         14.000000
75%         26.000000
max         28.000000
Name: Years_Open, dtype: float64

In [23]:
data.dtypes

Item_Fat_Content              object
Item_Identifier               object
Item_MRP                     float64
Item_Outlet_Sales            float64
Item_Type                     object
Item_Visibility              float64
Item_Weight                  float64
Outlet_Establishment_Year      int64
Outlet_Identifier             object
Outlet_Location_Type          object
Outlet_Size                   object
Outlet_Type                   object
file                          object
Years_Open                     int64
dtype: object

In [24]:
#One Hot Coding Categorical Dimensions
le = LabelEncoder()
data['Outlet'] = le.fit_transform(data['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type','Outlet']
le = LabelEncoder()
for i in var_mod:
    data[i] = le.fit_transform(data[i])

data = pd.get_dummies(data, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type','Outlet'])

In [25]:
data.dtypes

Item_Identifier               object
Item_MRP                     float64
Item_Outlet_Sales            float64
Item_Type                     object
Item_Visibility              float64
Item_Weight                  float64
Outlet_Establishment_Year      int64
Outlet_Identifier             object
file                          object
Years_Open                     int64
Item_Fat_Content_0             uint8
Item_Fat_Content_1             uint8
Outlet_Location_Type_0         uint8
Outlet_Location_Type_1         uint8
Outlet_Location_Type_2         uint8
Outlet_Size_0                  uint8
Outlet_Size_1                  uint8
Outlet_Size_2                  uint8
Outlet_Type_0                  uint8
Outlet_Type_1                  uint8
Outlet_Type_2                  uint8
Outlet_Type_3                  uint8
Outlet_0                       uint8
Outlet_1                       uint8
Outlet_2                       uint8
Outlet_3                       uint8
Outlet_4                       uint8
O

In [26]:
#Split back into test and train files
train = data.loc[data['file']=="train"]
test = data.loc[data['file']=="test"]

In [27]:
#Drop dimensions not needed
test.drop(['Outlet_Establishment_Year','Item_Outlet_Sales','file'],axis=1,inplace=True)
train.drop(['Outlet_Establishment_Year','file'],axis=1,inplace=True)

/Users/trevor/python-projects/analytics/venv/lib/python2.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [28]:
#Export modified files to csv
train.to_csv("data/train_modified.csv",index=False)
test.to_csv("data/test_modified.csv",index=False)